# First Project: Personal Carrer Agent

### And, Tool use.

**But first: introducing Pushover**

Pushover is a nifty tool for sending Push Notifications to your phone.  
It's super easy to set up and install!!

Simply visit [Pushover](https://pushover.net/) and sign up for a free account, and create your API key.  

Add to your `.env` file:

PUSHOVER_TOKEN=

PUSHOVER_USER=

And install the app in your mobile phone

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr


In [3]:
load_dotenv(override=True)
client=OpenAI()

In [4]:
pushover_user=os.getenv("PUSHOVER_USER")
pushover_token=os.getenv("PUSHOVER_TOKEN")
pushover_url="https://api.pushover.net/1/messages.json"

In [5]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [6]:
push("HEYY Neeraj!!")

Push: HEYY Neeraj!!


### Now we create some function and tools

In [26]:
def record_user_details(email, name="Name not Provided", notes="not Provided"):
    push(f"Recording interests from {name} with email {email} and notes {notes}")
    return {"Recorded": "OK"}

In [27]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"Recorded": "OK"}

In [28]:
record_user_details_json= {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "decription": " The user's name, if they provided it "
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
    
}

In [29]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answeres as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties":{
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required":["question"],
        "additionalProperties": False
    }
}

In [30]:
tools=[{"type": "function", "function": record_user_details_json},
      {"type": "function", "function": record_unknown_question_json}]

### Here we learn How to process and execute function requests

In [12]:
# this function can take a list of tool calls, and run them.
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name=tool_call.function.name
        arguments= json.loads(tool_call.function.arguments)
        print(f"Tool called : {tool_name}", flush=True)

        # if tool_name == "record_user_details":
        #     result= record_user_details(**arguments)
        # elif tool_name == "record_unknown_questions":
        #     result = record_unknown_questions(**arguments)

        tool=globals().get(tool_name)
        result= tool(**arguments) if tool else {}

        results.append({"role": "tool", "content": json.dumps(result),"tool_call_id": tool_call.id})
    return results


In [13]:
globals()["record_unknown_questions"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'Recorded': 'OK'}

In [14]:
reader=PdfReader("me/Neeraj Resume.pdf")
resume=""
for page in reader.pages:
    text= page.extract_text()
    if text:
        resume += text

with open("me/summary.txt", "r", encoding='utf-8') as f:
    summary=f.read()

name="Neeraj Sandhu"

In [18]:
system_prompt= f"You are acting as {name}. You are answering questions on {name}'s Website, \
    particularly questions related to {name}'s carrer, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know answer to any question , use your record_unknown_questions tool to record the question that you couldn't answer. \
    If user is enaging in discussion , try to steer them towards getting in touch via email; ask for their email and record it using your tools"

system_prompt+=f"\n\n ## Summary:\n{summary}\n\n##{resume}\n\n"
system_prompt+=f"with this context, please chat with the user, always staying in character as {name}"

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        # this is the call to LLm -- see that we pass in the tools json
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )
        
        finish_reason = response.choices[0].finish_reason

        #If the LLm want to call a tool we do that!

        if finish_reason=="tool_calls":
            message= response.choices[0].message
            tool_calls = message.tool_calls
            results= handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Tool called : record_unknown_question
Push: Recording Who is your favourite actor? asked that I couldn't answer
Tool called : record_user_details
Push: Recording interests from Name not Provided with email sandhuneeraj002@gmail.com and notes not Provided
